## Datos abiertos en México
Fue realmente interesante saber que en México se trabaja bastante por los datos abiertos y todo lo que esto conlleva, no solo abrir los datos al público en general, sino que dar la oportunidad de que la misma comunidad colabora creando cosas geniales como esta:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datosgobmx import client
import folium

In [3]:
# Importing parameters from sinaica, it has meteorological stations data 
parameters_request = client.makeCall('sinaica-parametros')

In [4]:
# The file that is returned in the past cell is in Json format, so we have to convert it in a Pandas Object}
valid_parameters = []
for element in parameters_request['results']:
    aux=pd.DataFrame.from_dict(element, orient='index').T
    valid_parameters.append(aux)
    
valid_parameters = pd.concat(valid_parameters, ignore_index=True)
valid_parameters['date-insert'] = pd.to_datetime(valid_parameters['date-insert'])

In [5]:
valid_parameters

,_id,parametro,date-insert
0,5c2aa3dfe2705c1932134299,CO,2018-12-31 23:18:55.923000+00:00
1,5c2aa3dfe2705c193213429a,O3,2018-12-31 23:18:55.923000+00:00
2,5c2aa3dfe2705c193213429b,PM10,2018-12-31 23:18:55.923000+00:00
3,5c2aa3dfe2705c193213429c,SO2,2018-12-31 23:18:55.923000+00:00
4,5c2aa3dfe2705c193213429d,NO2,2018-12-31 23:18:55.923000+00:00
5,5c2aa3dfe2705c193213429e,PM2.5,2018-12-31 23:18:55.923000+00:00
6,5c2aa3dfe2705c193213429f,TMP,2018-12-31 23:18:55.923000+00:00


De lo anterior, es interesante notar, que la organización de estos datos, es de las mejores con las que he trabajado en mi vida, estos parámetros lo que indican son, las distintas mediciones que hacen las estaciones de monitoreo, en este caso, hacen de monóxido de carbono, ozono, partículas menores a 10 micrómetros, dióxido de azufre, dióxido de nitrógeno, partículas menores a 2.5 micrómetros y temperatura respectivamente, más adelante se muestra qué tipos de datos son y lo que representan. (Es decir, si la temperatura está en grados Celsius o Kelvin o las partículas de Co2 están por partes por millón)

Pero ahora, lo anterior sólo fue para ver parámetros, véamos algunos datos, esto se hace de una manera parecida a lo de arriba:

In [6]:
estacionesSinaica1 = client.makeCall('sinaica-estaciones', {'pageSize':200})
stationSinaica = []
for element in estacionesSinaica1['results']:
    aux=pd.DataFrame.from_dict(element, orient='index').T
    stationSinaica.append(aux)
    
stationSinaica = pd.concat(stationSinaica, ignore_index=True)
#stationSinaica.to_csv("./data/station_data_clear.csv")

In [7]:
stationSinaica

,_id,lat,long,id,nombre,codigo,redesid
0,5cccf5bee2705c1932820580,21.8733,-102.321,31,CBTIS,CBT,30
1,5cccf5bee2705c1932820581,21.8464,-102.288,32,Secretaría de Medio Ambiente,SMA,30
2,5cccf5bee2705c1932820582,21.8838,-102.296,33,Centro,CEN,30
3,5cccf5bee2705c1932820583,32.6397,-115.506,39,COBACH,SPABC14,32
4,5cccf5bee2705c1932820584,32.6036,-115.486,41,CESPM,SPABC19,32
...,...,...,...,...,...,...,...
180,5cccf5bee2705c1932820635,31.7433,-106.432,428,Instituto de Ingeniería y Tecnología,IIT,150
181,5cccf5bee2705c1932820636,25.5719,-103.498,430,Parque La Esperanza,ESP,43
182,5cccf5bee2705c1932820637,19.4737,-99.0462,431,FES Aragón,FAR,119
183,5cccf5bee2705c1932820638,19.3456,-99.0094,432,Santiago Acahualtepec,SAC,119


Y eso, fácilmente lo podemos visualizar en un mapa, todo lo que aquí está escrito, se basa en la documentación: https://python-visualization.github.io/folium/, está bien bonito folium, por eso lo escogí.

In [8]:
# Center the map
cLat, cLon = 22.39, -101.73

# Create the map
foliumMap = folium.Map(location=[cLat, cLon], zoom_start=5, tiles="cartodb positron")

# Full the map with data above
for i, row in stationSinaica.iterrows():
    # Generating the map and the format
    popup_text = "<b>Nombre:</b> {} <br> <b>Latitud: </b> {:.5f} <br> <b> Longitud: </b> {:.5f} <br>".format(row.nombre.encode("latin-1"), row.lat, row.long)
    # Adding Marks
    folium.CircleMarker(location=[row.lat, row.long], radius=6, tooltip=popup_text, fill=True, fill_opacity=0.4).add_to(foliumMap)

In [9]:
# Deploy the map
foliumMap

Aquí podemos ver los lugares de donde se tienen mediciones, ahora, nuestro próximo objetivo será obtener esas mediciones y trabajar con ellas

In [10]:
measurementsinaica = client.makeCall('sinaica', {'pageSize':5000})
dataSinaica = []
for element in estacionesSinaica1['results']:
    aux=pd.DataFrame.from_dict(element, orient='index').T
    dataSinaica.append(aux)
    
dataSinaica = pd.concat(dataSinaica, ignore_index=True)
#dataSinaica.to_csv("./data/data_clear.csv", encoding="latin-1", mode="w")

In [11]:
dataSinaica

,_id,lat,long,id,nombre,codigo,redesid
0,5cccf5bee2705c1932820580,21.8733,-102.321,31,CBTIS,CBT,30
1,5cccf5bee2705c1932820581,21.8464,-102.288,32,Secretaría de Medio Ambiente,SMA,30
2,5cccf5bee2705c1932820582,21.8838,-102.296,33,Centro,CEN,30
3,5cccf5bee2705c1932820583,32.6397,-115.506,39,COBACH,SPABC14,32
4,5cccf5bee2705c1932820584,32.6036,-115.486,41,CESPM,SPABC19,32
...,...,...,...,...,...,...,...
180,5cccf5bee2705c1932820635,31.7433,-106.432,428,Instituto de Ingeniería y Tecnología,IIT,150
181,5cccf5bee2705c1932820636,25.5719,-103.498,430,Parque La Esperanza,ESP,43
182,5cccf5bee2705c1932820637,19.4737,-99.0462,431,FES Aragón,FAR,119
183,5cccf5bee2705c1932820638,19.3456,-99.0094,432,Santiago Acahualtepec,SAC,119


# Experiments

In [19]:
# Importing table
data = pd.read_csv("./data/cat_parametros.csv", header=1, encoding="latin-1", index_col=["id_parameter"])

In [20]:
data

,cve_param,nom_param,unidades_param
id_parameter,,,
1,O3,Ozono,1
2,NO2,Dióxido de nitrógeno,1
3,NO,Óxido nítrico,1
4,NOx,Óxidos de nitrógeno,1
5,CO,Monóxido de carbono,15
6,SO2,Dióxido de azufre,1
7,PM10,Partículas menores a 10 micrómetros,2
8,PM2.5,Partículas menores a 2.5 micrómetros,2
9,PMco,Partículas coarse,2
